In [118]:
import pandas as pd
import datetime as dt

# Leitura dos arquivos

In [119]:
df_configEmpresa = pd.read_csv('bases/Configuracao_Empresas.csv', sep=';').sort_values('prioridade')
df_fasesProjetos = pd.read_csv('bases/fases_projetos.csv', sep=';')

In [120]:
df_configEmpresa

,empresa,prioridade,grupo,possui trimestral,deadline,horas,senior,staff 3,staff 2,trainee,fase_planejamento,fase_interim,fase_final,fase_trimestral
0,SUL AMERICA,1,1,Yes,Fev / Mar,18201,4,0,3,2,13,15,16,4
1,REDE DOR,2,1,Yes,Fev / Mar,18850,4,0,3,2,13,15,16,4
2,TIM,3,1,Yes,Fev,16400,3,0,3,2,13,15,16,4
3,GLOBO - GLOBO,4,1,Yes,Mar,11550,2,0,3,2,13,15,16,4
4,BROOKFIELD ENERGY (ELERA),5,1,Yes,Fev / Mar,12000,2,0,3,2,13,15,16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,CALEFFI BRASIL AUP,154,11,No,Fev,200,0,1,1,1,0,1,3,0
154,Planeta / DeAgostini,155,11,No,Mar,200,0,1,1,1,0,1,3,0
155,TI LATAM AUP,156,11,No,Abr,200,0,1,1,1,0,1,3,0
156,ALLSEAS,157,11,No,Jun,160,0,1,1,1,0,1,3,0


In [102]:
df_fasesProjetos

,phase,code,part,start,end,interval
0,Análise Trimetral - 1,fase_trimestral,T,2024-04-13,2024-05-11,4
1,Análise Trimetral - 2,fase_trimestral,T,2023-07-15,2023-08-12,4
2,Análise Trimetral - 3,fase_trimestral,T,2023-10-14,2023-11-11,4
3,Final,fase_final,NaN,2024-01-13,2024-03-30,11
4,Planejamento,fase_planejamento,G,2024-04-13,2024-06-30,13
5,Planejamento,fase_planejamento,P,2023-08-12,2023-11-12,13
6,Interim,fase_interim,G,2023-10-15,2024-01-15,13
7,Interim,fase_interim,P,2023-10-15,2024-01-15,13
8,Férias,fase_ferias,F,2023-12-23,2024-01-05,1
9,Férias,fase_ferias,F,2024-06-15,2024-06-28,1


Etapas do desenvolvimento:
1. Carga das bases
    1. Planejamento
    1. Bribous
1. Obter todos as datas de inicio da semana
1. Loop [Por cliente]
     1. Obter os clientes ordenados por prioridade
     1. Obter datas de inicio e fim de cada uma das fases do cliente
     1. Adicionar as horas dos recursos dentro do periodo indicado
         1. Identificar a quantidade de semanas utilizadas em cada uma das fases
         1. Alocar os recursos de acordo com o periodo com menor desvio padrão

# Métodos

## Obter todas as segunda-feiras do Fiscal Year

In [121]:
global_freq = 'W-SAT'

In [122]:
def getSpecificDayofWeek (startDate='2023-07-15', endDate='2024-07-15', freq=global_freq):
    """
        W-SUN, W-MON, W-TUE, W-WED, W-THU, W-FRI, W-SAT
    """
    # Criação de todas as datas entre o startDate e endDate
    dates = pd.date_range(start=startDate, end=endDate, freq=freq)
    
    return dates

def validateWeekAlocation(row_company, week, phase_start, phase_end, vacation_weeks):
    """
        Função de validação da semana para verificar se é possível realizar a alocação do recurso ou não.
    """    
    
    if not pd.Timestamp(phase_start) <= week:
        print("Invalid: Semana não esta no range de datas desta fase.")
        return -1
    
    if not week <= pd.Timestamp(phase_end):
        print("Invalid: Semana não esta no range de datas desta fase.")
        return -2
    
    if row_company[week] > 40:
        print("Invalid: Valor acima de 40.")
        return 0
    
    if week in vacation_weeks:
        print("Invalid: Semana de período de férias.")
        return 0
    
    return 1

def getWeekLowestHours(df_company, weeks=pd.DatetimeIndex([])):
    
    if weeks.empty:
        return None
    
    #  Calcular o desvio padrão de cada semana
    total_sum = df_company[weeks].sum()
    
    # Identificar qual semana tem o menor desvio padrão
    min_sum_week = total_sum.idxmin()
    
    return min_sum_week


def getVacationPeriod():
    
    df_phase = pd.read_csv('bases/fases_projetos.csv', sep=';')
    code='fase_ferias'
    
    filter_phases_vacation = df_phase[df_phase['code'] == 'fase_ferias']
    
    all_dates = []
    
    for index, row in filter_phases_vacation.iterrows():
        data_range = pd.date_range(start=row['start'], periods=2, freq='W-SAT')
        all_dates.extend(data_range)

    return all_dates

# Preparando tabelas auxiliar de datas

# Preparando tabela auxiliar das empresas

# Preparar o arquivo de saida

In [123]:
def filterCompanyPhases(rowCompany, df_phase):
    """
        Função para filtrar as phases de um empresa para realizar a alocação dos recursos.
    """
    #Definir se é empresa Grande ou Pequena
    values_filter = ['P'] if rowCompany['horas'] > 2000 else ['G']
    
    # Remover as fases de Férias
    values_filter.append('F')
    
    # Caso não tenha trimestral remover estas fase tambem
    if rowCompany['possui trimestral'] == 'No':
        values_filter.append('T')
    
    return df_phase[~df_phase['part'].isin([values_filter])]

def getValidWeekLowestHours(df_company, row_company, week_min, week_max, phase_start, phase_end, vacation_weeks):
        
    week_min_valid = validateWeekAlocation(row_company, week_min, phase_start, phase_end, vacation_weeks)
    while week_min_valid != 1 and week_min >= pd.Timestamp(phase_start):
        week_min -= dt.timedelta(weeks=1)
        week_min_valid = validateWeekAlocation(row_company, week_min, phase_start, phase_end, vacation_weeks)
    
    week_max_valid = validateWeekAlocation(row_company, week_min, phase_start, phase_end, vacation_weeks)
    while week_max_valid != 1 and week_max <= pd.Timestamp(phase_end):
        week_max += dt.timedelta(weeks=1)
        week_max_valid = validateWeekAlocation(row_company, week_max, phase_start, phase_end, vacation_weeks)
        
    weeks_compare = pd.DatetimeIndex([])
    
    if week_min_valid not in [-1, -2]:
        weeks_compare = weeks_compare.union([week_min])
        
    if week_max_valid not in [-1, -2]:
        weeks_compare = weeks_compare.union([week_max])
    
    return getWeekLowestHours(df_company, weeks_compare), week_min, week_max

def insertPhaseWeek(df_company, index_company, row_company, row_phase, vacation_weeks):
    """
        Função de validação da semana para verificar se é possível realizar a alocação do recurso ou não.

        - Obter a empresa
        - Obter a lista de fases
        - Filtrar as fases especificas para a empresa passada 
        - Para cada fase verificar a data de inicio e fim
        - 
    """
    
    phase_code = row_phase['code']
    phase_interval = row_phase['interval']
    phase_start = row_phase['start']
    phase_end = row_phase['end']

    if phase_code == 'fase_ferias':
        return

    total_period = phase_interval if row_company[phase_code] > phase_interval else row_company[phase_code]
    
    # Obtem a primeira semana com o menor desvio padrão para alocar os recursos
    first_week = getWeekLowestHours(df_company, getSpecificDayofWeek(phase_start, phase_end))    
    
    week_current = first_week
    week_max = first_week
    week_min = first_week
    
    for i in range(total_period):
        
        while (week_current != None):
            
            week_current, week_min, week_max = getValidWeekLowestHours(df_company, row_company, week_min, week_max
                                                                       , phase_start, phase_end, vacation_weeks)
    
            if week_current != None:
                df_company.loc[index_company, week_current] += 40
                break
                

def replaceCurrentWeek(week_current, week_min, week_max):
    if week_current == week_min:
        week_min = week_min - dt.timedelta(weeks=1)
    
    if week_current == week_max:
        week_max = week_max + dt.timedelta(weeks=1)
    
    return week_min, week_max
    
                
def getOutputStructure(df_company, start_date, end_date):
    
    df_list_days = pd.DataFrame(getSpecificDayofWeek(start_date, end_date), columns=['data'])
    
    df_list_days = pd.DataFrame([], columns=df_lstDias['data'])
    
    df_result = pd.concat([df_company, df_list_days], axis=1).fillna(0)
    
    return df_result

def arrangeAllResources(df_company, df_phases, start_date, end_date):
    
    vacation_weeks = getVacationPeriod()
    
    df_result = getOutputStructure(df_company, start_date, end_date)
    
    for index_result, row_result in df_result.iterrows():
    
        df_company_phase = filterCompanyPhases(row_result, df_phases)
        
        for index_phase, row_phase in df_company_phase.iterrows():
        
            insertPhaseWeek(df_result, index_result, row_result, row_phase, vacation_weeks)
    
    for col in df_result.columns:
        if isinstance(col, dt.datetime):
            df_result.rename(columns={col: col.strftime('%Y-%m-%d')}, inplace=True)
    
    df_result.to_excel('Resultado.xlsx', index=False)


In [124]:
arrangeAllResources(df_configEmpresa, df_fasesProjetos, '2023-07-15', '2024-07-15')

Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Semana de período de férias.
Invalid: Sem

PermissionError: [Errno 13] Permission denied: 'Resultado.xlsx'

df_result = pd.concat([df_lstEmpresas, df_diasSemana], axis=1).fillna(0)

weeksVacation = 
df_result = includeVacationPeriod(df_result, )

# Para cada cliente na tabela de Empresas
for indexEmp, rowEmp in df_result.iterrows():
    
    valueFilter = ['P'] if rowEmp['horas'] > 2000 else ['G']
    # Possui análise trimestral?
    if rowEmp['possui trimestral'] == 'No':
        valueFilter.append('T')
    df_fases = df_fasesProjetos[~df_fasesProjetos['Part'].isin([valueFilter])]
                                      
    # Para cada período na tabela fase alocar o recurso no dataframe
    print("==========================================================")
    for indexFase, rowFase in df_fases.iterrows():
        periodo = 2
        qtd_horas = -999 # Removendo todas as horas deste periodo.
        if rowFase['codigo'] in df_result.columns:
            qtd_horas = 40
            periodo = rowEmp[rowFase['codigo']]
            if rowEmp[rowFase['codigo']] > rowFase['Interval']:
                periodo = rowFase['Interval']
            
        # Obtem a primeira semana com o menor desvio padrão para alocar os recursos
        firstWeek = getWeekLowestStDevs(df_result, getSpecificDayofWeek(startDate=rowFase['Inicio'], endDate=rowFase['Fim']))     
                
        # Obtem lista de semanas com o periodo indicado pelo grupo da empresa para a respectiva fase
        weeks = pd.date_range(start=firstWeek, periods=periodo, freq=global_freq)
        
        # Verificar a quantidade de semanas que passou alem da data fim da fase
        wCount = sum(1 for w in weeks if w > dt.datetime.strptime(rowFase['Fim'], "%Y-%m-%d"))
        
        # Caso tenha mais de um periodo devemos remover estes periodos e adicionar para o inicio da alocação
        if wCount > 0:
            print("Empresa: ",rowEmp['empresa']," - Fase: ",rowFase['codigo']," - Inicio: ",rowFase['Inicio']," - Fim: ",rowFase['Fim'])
            print("firstWeek: ", firstWeek)
            
            print(weeks)
            print(wCount)
            # Obtem a primeira data que deveria iniciar a alocação para evitar que ultrapasse a data final da fase 
            firstWeek = pd.date_range(end=firstWeek - dt.timedelta(days=1), periods=wCount, freq=global_freq)[0]

            print("firstWeek Depois: ", firstWeek)
            weeks = pd.date_range(start=firstWeek, periods=periodo, freq=global_freq)
        
            print(weeks)
        
        for w in weeks:
            
            df_result.loc[indexEmp, w] += qtd_horas
    print("==========================================================")

# Salvando o Resultado